In [1]:
import pandas as pd  #pacotes
import numpy as np   #calculo numerico
import glob          #listas
#Codigo para declarar um pyarrow parquet - Willian - 29/05/2018
import pyarrow.parquet as pq
#uproot serve para ler a partir o python um arquivo em tree root
#import uproot
import os, sys
from os import path
import datetime
import matplotlib as mp
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.optimize import curve_fit
from matplotlib.colors import BoundaryNorm
%matplotlib inline

## MEDIDAS PARA PIETRO (ANALISE BAYSENIANA) ON e OFF 72h CONTINUO

In [2]:
%run def_functions.ipynb
%run def_functions_hits.ipynb
%run def_functions_fits.ipynb


## Salvar e carregar Listas ON / OFF

## Função para processar lista

### Contagem de período de 168h (7dias) dentro do tempo (8-50us), fora do tempo (58-100us)
### Para análise Baysiana do Pietro

In [3]:
def fnt_process_list_excess (p_list_arq, p_list_period):

    p_list_in = []
    p_list_out = []

    print('sinal - 8us < t < 50us')
    print('ruído - 58us < t < 100us')

    for ind, list_arq in enumerate(p_list_arq):
        p_total_in = 0
        p_total_out = 0
        print(p_list_period[ind])
        for file in p_list_arq[ind]:
           # print('  >> '+file)
            ##yyyy=int(sListDt[5:9]); mm=int(sListDt[9:11]); dd=int(sListDt[11:13])
            
            df = pd.read_parquet(file)
            dff_in = df[(df['p_satured']==0)&(df['d_satured']==0)
                            &(fnt_pe2MeV(df['p_Total_pe'])>=3)&(fnt_pe2MeV(df['p_Total_pe'])<=10)
                            &(fnt_pe2MeV(df['d_Total_pe'])>=1.6)&(fnt_pe2MeV(df['d_Total_pe'])<=7.12)
                            &(df['d_timeInverval']>=8)&(df['d_timeInverval']<=50)  ##  o que muda é o intervalo 8 a 50us
                            &(df['d_mPMTs']>=12)
                       ]
            dff_out = df[(df['p_satured']==0)&(df['d_satured']==0)
                            &(fnt_pe2MeV(df['p_Total_pe'])>=3)&(fnt_pe2MeV(df['p_Total_pe'])<=10)
                            &(fnt_pe2MeV(df['d_Total_pe'])>=1.6)&(fnt_pe2MeV(df['d_Total_pe'])<=7.12)
                            &(df['d_timeInverval']>=58)&(df['d_timeInverval']<=100)  ##  o que muda é o intervalo 58 a 100us
                            &(df['d_mPMTs']>=12)
                        ]
                        
            p_total_in += dff_in.count()[0]
            p_total_out += dff_out.count()[0]
        
        p_list_in.append(p_total_in)
        p_list_out.append(p_total_out)

        #print(p_totalPositrons)
        #break


    return p_list_in, p_list_out

In [12]:
def fnt_process_list_excess_58 (p_list_arq, p_list_period):

    p_list_out = []

    print('ruído - t > 58us (mult 12)')

    for ind, list_arq in enumerate(p_list_arq):
        p_total_out = 0
        print(p_list_period[ind])
        for file in p_list_arq[ind]:
           # print('  >> '+file)
            ##yyyy=int(sListDt[5:9]); mm=int(sListDt[9:11]); dd=int(sListDt[11:13])

            df = pd.read_parquet(file)
            dff_out = df[(df['p_satured']==0)&(df['d_satured']==0)
                            &(fnt_pe2MeV(df['p_Total_pe'])>=3)&(fnt_pe2MeV(df['p_Total_pe'])<=10)
                            &(fnt_pe2MeV(df['d_Total_pe'])>=1.6)&(fnt_pe2MeV(df['d_Total_pe'])<=7.12)
                            &(df['d_timeInverval']>=58)  ##  o que muda é o intervalo maior que 58us
                            &(df['d_mPMTs']>=12)
                        ]

            p_total_out += dff_out.count()[0]
        
        p_list_out.append(p_total_out)

        #print(p_totalPositrons)
        #break


    return p_list_out

## Processamento do período

## Parte 1: período de 15/06/2020 a 08/09/2020

In [5]:
print('Inicio',datetime.datetime.now())

list_dtin_168h=[]
list_dtfn_168h=[]
list_in=[]
list_out=[]

list_arq_168h, list_dt_168h = find_interval_hour_pair('2020-06-15','2020-09-14', 168) #'2020-12-31', 168)
print(list_dt_168h)
list_in, list_out = fnt_process_list_excess(list_arq_168h, list_dt_168h)

#print('\n', list_dt_168h)
#print('\n\n', list_in)
#print('\n\n', list_out)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-27 12:23:00.113520
found data/20200615
found data/20200616
found data/20200617
found data/20200618
found data/20200619
found data/20200620
found data/20200621
found data/20200622
found data/20200623
found data/20200624
found data/20200625
found data/20200626
found data/20200627
found data/20200628
found data/20200629
found data/20200630
found data/20200701
found data/20200702
found data/20200703
found data/20200704
found data/20200705
found data/20200706
found data/20200707
found data/20200708
found data/20200709
found data/20200710
found data/20200711
found data/20200712
found data/20200713
found data/20200714
found data/20200715
found data/20200716
found data/20200717
found data/20200718
found data/20200719
found data/20200720
found data/20200721
found data/20200722
found data/20200723
found data/20200724
found data/20200725
found data/20200726
found data/20200727
found data/20200728
found data/20200729
found data/20200730
found data/20200731
found data/20200801
found 

In [ ]:
list_tot = [list_dt_168h, list_in, list_out]
print(list_tot)

## Parte 1 mas com t > 58us

In [13]:
print('Inicio',datetime.datetime.now())

list_dtin_168h_58=[]
list_dtfn_168h_58=[]
list_out_58=[]

list_arq_168h_58, list_dt_168h_58 = find_interval_hour_pair('2020-06-15','2020-09-14', 168) 
print(list_dt_168h_58)
list_out_58 = fnt_process_list_excess_58(list_arq_168h_58, list_dt_168h_58)

print('\n', list_dt_168h_58)
print('\n\n', list_out_58)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-27 23:37:06.965192
found data/20200615
found data/20200616
found data/20200617
found data/20200618
found data/20200619
found data/20200620
found data/20200621
found data/20200622
found data/20200623
found data/20200624
found data/20200625
found data/20200626
found data/20200627
found data/20200628
found data/20200629
found data/20200630
found data/20200701
found data/20200702
found data/20200703
found data/20200704
found data/20200705
found data/20200706
found data/20200707
found data/20200708
found data/20200709
found data/20200710
found data/20200711
found data/20200712
found data/20200713
found data/20200714
found data/20200715
found data/20200716
found data/20200717
found data/20200718
found data/20200719
found data/20200720
found data/20200721
found data/20200722
found data/20200723
found data/20200724
found data/20200725
found data/20200726
found data/20200727
found data/20200728
found data/20200729
found data/20200730
found data/20200731
found data/20200801
found 

## Parte 2: segundo período

In [6]:
print('Inicio',datetime.datetime.now())

list_dtin_168h_2=[]
list_dtfn_168h_2=[]
list_in_2=[]
list_out_2=[]

list_arq_168h_2, list_dt_168h_2 = find_interval_hour_pair('2020-09-24','2020-12-31', 168) #'2020-12-31', 168)
print(list_dt_168h_2)
list_in_2, list_out_2 = fnt_process_list_excess(list_arq_168h_2, list_dt_168h_2)

#print('\n', list_dt_168h)
#print('\n\n', list_in)
#print('\n\n', list_out)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-27 15:26:28.698054
found data/20200924
found data/20200925
found data/20200926
found data/20200927
found data/20200928
found data/20200929
found data/20200930
found data/20201001
found data/20201002
found data/20201003
found data/20201004
found data/20201005
found data/20201006
found data/20201007
found data/20201008
found data/20201009
found data/20201010
found data/20201011
found data/20201012
found data/20201013
found data/20201014
found data/20201015
found data/20201016
found data/20201017
found data/20201018
found data/20201019
found data/20201020
found data/20201021
found data/20201022
found data/20201023
found data/20201024
found data/20201025
found data/20201026
found data/20201027
found data/20201028
found data/20201029
found data/20201030
found data/20201031
found data/20201101
found data/20201102
found data/20201103
found data/20201104
found data/20201105
found data/20201106
found data/20201107
found data/20201108
found data/20201109
found data/20201110
found 

## Parte 2 com t > 58us

In [14]:
print('Inicio',datetime.datetime.now())

list_dtin_168h_58_2=[]
list_dtfn_168h_58_2=[]
list_out_58_2=[]

list_arq_168h_58_2, list_dt_168h_58_2 = find_interval_hour_pair('2020-09-24','2020-12-31', 168) 
print(list_dt_168h_58_2)
list_out_58_2 = fnt_process_list_excess_58(list_arq_168h_58_2, list_dt_168h_58_2)

#print('\n', list_dt_168h_58)
#print('\n\n', list_out_58)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-28 02:49:08.401067
found data/20200924
found data/20200925
found data/20200926
found data/20200927
found data/20200928
found data/20200929
found data/20200930
found data/20201001
found data/20201002
found data/20201003
found data/20201004
found data/20201005
found data/20201006
found data/20201007
found data/20201008
found data/20201009
found data/20201010
found data/20201011
found data/20201012
found data/20201013
found data/20201014
found data/20201015
found data/20201016
found data/20201017
found data/20201018
found data/20201019
found data/20201020
found data/20201021
found data/20201022
found data/20201023
found data/20201024
found data/20201025
found data/20201026
found data/20201027
found data/20201028
found data/20201029
found data/20201030
found data/20201031
found data/20201101
found data/20201102
found data/20201103
found data/20201104
found data/20201105
found data/20201106
found data/20201107
found data/20201108
found data/20201109
found data/20201110
found 

## Semana isolada de 141h: 08/09/2020 - 13/09/2020

In [7]:
print('Inicio',datetime.datetime.now())

list_dtin_168h_1i=[]
list_dtfn_168h_1i=[]
list_in_1i=[]
list_out_1i=[]

list_arq_168h_1i, list_dt_168h_1i = find_interval_hour_pair('2020-09-08','2020-09-13', 141)
print(list_dt_168h_1i)
list_in_1i, list_out_1i = fnt_process_list_excess(list_arq_168h_1i, list_dt_168h_1i)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-27 18:44:18.418006
found data/20200908
found data/20200909
found data/20200910
found data/20200911
found data/20200912
found data/20200913
[['2020-09-08', '2020-09-13']]
sinal - 8us < t < 50us
ruído - 58us < t < 100us
['2020-09-08', '2020-09-13']
Fim 2024-09-27 18:57:13.645102


## Semana isolada de 141h: 08/09/2020 - 13/09/2020 com t > 58us

In [15]:
print('Inicio',datetime.datetime.now())

list_dtin_168h_58_1i=[]
list_dtfn_168h_58_1i=[]
list_out_58_1i=[]

list_arq_168h_58_1i, list_dt_168h_58_1i = find_interval_hour_pair('2020-09-08','2020-09-13', 141)
print(list_dt_168h_58_1i)
list_out_58_1i = fnt_process_list_excess_58(list_arq_168h_58_1i, list_dt_168h_58_1i)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-28 06:12:25.792151
found data/20200908
found data/20200909
found data/20200910
found data/20200911
found data/20200912
found data/20200913
[['2020-09-08', '2020-09-13']]
ruído - t > 58us (mult 12)
['2020-09-08', '2020-09-13']
Fim 2024-09-28 06:25:50.659994


## Semana isolada de 164h: 25/12/2020 - 31/12/2020

In [8]:
print('Inicio',datetime.datetime.now())

list_dtin_168h_2i=[]
list_dtfn_168h_2i=[]
list_in_2i=[]
list_out_2i=[]

list_arq_168h_2i, list_dt_168h_2i = find_interval_hour_pair('2020-12-25','2020-12-31', 164)
print(list_dt_168h_2i)
list_in_2i, list_out_2i = fnt_process_list_excess(list_arq_168h_2i, list_dt_168h_2i)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-27 18:57:13.731126
found data/20201225
found data/20201226
found data/20201227
found data/20201228
found data/20201229
found data/20201230
found data/20201231
[['2020-12-25', '2020-12-31']]
sinal - 8us < t < 50us
ruído - 58us < t < 100us
['2020-12-25', '2020-12-31']
Fim 2024-09-27 19:11:54.709465


## Semana isolada de 164h: 25/12/2020 - 31/12/2020 com t >58us

In [16]:
print('Inicio',datetime.datetime.now())

list_dtin_168h_58_2i=[]
list_dtfn_168h_58_2i=[]
list_out_58_2i=[]

list_arq_168h_58_2i, list_dt_168h_58_2i = find_interval_hour_pair('2020-12-25','2020-12-31', 164)
print(list_dt_168h_58_2i)
list_out_58_2i = fnt_process_list_excess_58(list_arq_168h_58_2i, list_dt_168h_58_2i)
        
print('Fim',datetime.datetime.now())

Inicio 2024-09-28 06:25:50.733221
found data/20201225
found data/20201226
found data/20201227
found data/20201228
found data/20201229
found data/20201230
found data/20201231
[['2020-12-25', '2020-12-31']]
ruído - t > 58us (mult 12)
['2020-12-25', '2020-12-31']
Fim 2024-09-28 06:41:08.260698


## Result in print

In [9]:
for ind in range(len(list_in)):
    print('Record ('+str(ind+1)+'): '+list_dt_168h[ind][0]+' - '+list_dt_168h[ind][1]+' >> in: '+str(list_in[ind])+', out: '+str(list_out[ind]))
    
for ind_1i in range(len(list_in_1i)):
    print('Record ('+str(ind_1i+ind+2)+'): '+list_dt_168h_1i[ind_1i][0]+' - '+list_dt_168h_1i[ind_1i][1]+' >> in: '+str(list_in_1i[ind_1i])+', out: '+str(list_out_1i[ind_1i]))
    
for ind_2 in range(len(list_in_2)):
    print('Record ('+str(ind_2+ind_1i+ind+3)+'): '+list_dt_168h_2[ind_2][0]+' - '+list_dt_168h_2[ind_2][1]+' >> in: '+str(list_in_2[ind_2])+', out: '+str(list_out_2[ind_2]))
    
for ind_2i in range(len(list_in_2i)):
    print('Record ('+str(ind_2+ind_1i+ind+4)+'): '+list_dt_168h_2i[ind_2i][0]+' - '+list_dt_168h_2i[ind_2i][1]+' >> in: '+str(list_in_2i[ind_2i])+', out: '+str(list_out_2i[ind_2i]))

print('Fim dos cálculos',datetime.datetime.now())

Record (1): 2020-06-15 - 2020-06-22 >> in: 290083, out: 69997
Record (2): 2020-06-22 - 2020-06-29 >> in: 300194, out: 72503
Record (3): 2020-06-29 - 2020-07-06 >> in: 300615, out: 72562
Record (4): 2020-07-06 - 2020-07-13 >> in: 304790, out: 73078
Record (5): 2020-07-13 - 2020-07-20 >> in: 289274, out: 68776
Record (6): 2020-07-20 - 2020-07-27 >> in: 293423, out: 70251
Record (7): 2020-07-27 - 2020-08-03 >> in: 282796, out: 67780
Record (8): 2020-08-03 - 2020-08-10 >> in: 283150, out: 68986
Record (9): 2020-08-10 - 2020-08-17 >> in: 300899, out: 72827
Record (10): 2020-08-17 - 2020-08-25 >> in: 291846, out: 68500
Record (11): 2020-08-25 - 2020-09-01 >> in: 298127, out: 69420
Record (12): 2020-09-01 - 2020-09-08 >> in: 303175, out: 66291
Record (13): 2020-09-08 - 2020-09-13 >> in: 257279, out: 55903
Record (14): 2020-09-24 - 2020-10-01 >> in: 303144, out: 66820
Record (15): 2020-10-01 - 2020-10-08 >> in: 306551, out: 68402
Record (16): 2020-10-08 - 2020-10-15 >> in: 303524, out: 66078
R

## Result com t > 58us

In [60]:
print(list_out_58_1i)
print(list_dt_168h_58_1i)

print(list_out_58_2i)
print(list_dt_168h_58_2i)

([27200], [5059])
[['2020-09-08', '2020-09-13']]
[652514]
[['2020-12-25', '2020-12-31']]


In [63]:
for ind in range(len(list_out_58)):
    print('Record ('+str(ind+1)+'): '+list_dt_168h_58[ind][0]+' -  out: '+str(list_out_58[ind]))
    
for ind_1i in range(len(list_out_58_1i)):
    print('Record ('+str(ind_1i+ind+2)+'): '+list_dt_168h_58_1i[ind_1i][0]+' -  out: '+str(list_out_58_1i[ind_1i]))
    
for ind_2 in range(len(list_out_58_2)):
    print('Record ('+str(ind_2+ind_1i+ind+3)+'): '+list_dt_168h_58_2[ind_2][0]+' -  out: '+str(list_out_58_2[ind_2]))
    
for ind_2i in range(len(list_out_58_2i)):
    print('Record ('+str(ind_2+ind_1i+ind+4)+'): '+list_dt_168h_58_2i[ind_2i][0]+' -  out: '+str(list_out_58_2i[ind_2i]))


Record (1): 2020-06-15 -  out: 619004
Record (2): 2020-06-22 -  out: 639158
Record (3): 2020-06-29 -  out: 644938
Record (4): 2020-07-06 -  out: 651582
Record (5): 2020-07-13 -  out: 622044
Record (6): 2020-07-20 -  out: 631563
Record (7): 2020-07-27 -  out: 611496
Record (8): 2020-08-03 -  out: 603628
Record (9): 2020-08-10 -  out: 650772
Record (10): 2020-08-17 -  out: 631760
Record (11): 2020-08-25 -  out: 647779
Record (12): 2020-09-01 -  out: 637205
Record (13): 2020-09-08 -  out: 538792
Record (14): 2020-09-24 -  out: 657445
Record (15): 2020-10-01 -  out: 677880
Record (16): 2020-10-08 -  out: 663242
Record (17): 2020-10-15 -  out: 641232
Record (18): 2020-10-22 -  out: 680376
Record (19): 2020-10-29 -  out: 656195
Record (20): 2020-11-05 -  out: 643345
Record (21): 2020-11-12 -  out: 663621
Record (22): 2020-11-19 -  out: 636891
Record (23): 2020-11-27 -  out: 666759
Record (24): 2020-12-04 -  out: 650330
Record (25): 2020-12-11 -  out: 657376
Record (26): 2020-12-18 -  out: 65

## Result in print com out > 58us

In [17]:
for ind in range(len(list_in)):
    print('Record ('+str(ind+1)+'): '+list_dt_168h[ind][0]+' - '+list_dt_168h[ind][1]+' >> in: '+str(list_in[ind])+', out: '+str(list_out_58[ind]))
    
for ind_1i in range(len(list_in_1i)):
    print('Record ('+str(ind_1i+ind+2)+'): '+list_dt_168h_1i[ind_1i][0]+' - '+list_dt_168h_1i[ind_1i][1]+' >> in: '+str(list_in_1i[ind_1i])+', out: '+str(list_out_58_1i[ind_1i]))
    
for ind_2 in range(len(list_in_2)):
    print('Record ('+str(ind_2+ind_1i+ind+3)+'): '+list_dt_168h_2[ind_2][0]+' - '+list_dt_168h_2[ind_2][1]+' >> in: '+str(list_in_2[ind_2])+', out: '+str(list_out_58_2[ind_2]))
    
for ind_2i in range(len(list_in_2i)):
    print('Record ('+str(ind_2+ind_1i+ind+4)+'): '+list_dt_168h_2i[ind_2i][0]+' - '+list_dt_168h_2i[ind_2i][1]+' >> in: '+str(list_in_2i[ind_2i])+', out: '+str(list_out_58_2i[ind_2i]))

print('Fim dos cálculos (in x out(>58us))',datetime.datetime.now())

Record (1): 2020-06-15 - 2020-06-22 >> in: 290083, out: 7490227
Record (2): 2020-06-22 - 2020-06-29 >> in: 300194, out: 7665584
Record (3): 2020-06-29 - 2020-07-06 >> in: 300615, out: 7711305
Record (4): 2020-07-06 - 2020-07-13 >> in: 304790, out: 7733262
Record (5): 2020-07-13 - 2020-07-20 >> in: 289274, out: 7438325
Record (6): 2020-07-20 - 2020-07-27 >> in: 293423, out: 7534427
Record (7): 2020-07-27 - 2020-08-03 >> in: 282796, out: 7376137
Record (8): 2020-08-03 - 2020-08-10 >> in: 283150, out: 7416514
Record (9): 2020-08-10 - 2020-08-17 >> in: 300899, out: 7758711
Record (10): 2020-08-17 - 2020-08-25 >> in: 291846, out: 7457013
Record (11): 2020-08-25 - 2020-09-01 >> in: 298127, out: 7444900
Record (12): 2020-09-01 - 2020-09-08 >> in: 303175, out: 6987591
Record (13): 2020-09-08 - 2020-09-13 >> in: 257279, out: 5868817
Record (14): 2020-09-24 - 2020-10-01 >> in: 303144, out: 7096703
Record (15): 2020-10-01 - 2020-10-08 >> in: 306551, out: 7244451
Record (16): 2020-10-08 - 2020-10-

## Non